# Keras Pipeline for Posture Detection

## Defining transformers  

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
#from category_encoders.one_hot import OneHotEncoder
#from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer
#from category_encoders.ordinal import OrdinalEncoder
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings("ignore")
import ipytest.magics
import pytest
# set the file name (required)
__file__ = 'drone_pos_model.ipynb'

In [2]:
class Shuffler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        x=x.loc[np.random.permutation(x.index)]
        
        return x
############################################################################################
class XCentralizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, x_columns):
        self.x_columns = x_columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shift=x[["rightShoulder_x","leftShoulder_x","leftHip_x","rightHip_x"]].sum(axis=1)/4
        for col in self.x_columns:
            x[col] = x[col] - shift
        return x
############################################################################################
    
class YCentralizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, y_columns):
        self.y_columns = y_columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shift=x[["rightShoulder_y","leftShoulder_y","leftHip_y","rightHip_y"]].sum(axis=1)/4
        for col in list(set(self.y_columns)-set(["label"])):
            x[col] = x[col] - shift
        return x
############################################################################################

class YScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x): #x is df
        shoulder_y = x[["rightShoulder_y","leftShoulder_y"]].sum(axis=1)/2
        hip_y = x[["leftHip_y","rightHip_y"]].sum(axis=1)/2
        y_dist = hip_y - shoulder_y
        
        for col in list(set(x.columns)-set(["label"])):
            x[col] /= y_dist
        return x

## Data inspecting

In [4]:
#!pwd
#df = pd.read_csv("video_001.csv", delimiter=',')
# Christian's video is less noisy. Therefore I only train the model with his data at the moment. 
# acc increased 5 % taking his video camparing to all videos.
#path = "all_videos_posture_steptime50_checksum8160"
#path = "video_Christian_posture_steptime50_checksum8160"
path = "video_all_posture_steptime50_checksum8160"
df = pd.read_csv("../data/"+ path + ".csv",low_memory=False)
#df=df.drop([5557], axis=0)
#type(df.leftShoulder_x)
#df.info()
#print(df.dtypes)
#print()
print('labels: \n', df['label'].unique())
# we have a mix of categorical, numeric, and string data.
df.head(3)

labels: 
 [1 3 4 2 0]


,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftElbow_x,leftElbow_y,rightElbow_x,rightElbow_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,leftHip_x,leftHip_y,rightHip_x,rightHip_y,label
0,0.49250,0.18750,0.4000,0.18250,0.5050,0.26000,0.34375,0.19500,0.50875,0.33875,0.26625,0.16875,0.46500,0.34375,0.41125,0.34625,1
1,0.49250,0.18875,0.4025,0.18625,0.5075,0.25875,0.33250,0.19750,0.50000,0.33750,0.27625,0.17500,0.46750,0.33625,0.40875,0.33750,1
2,0.49125,0.19000,0.4025,0.17875,0.5050,0.26125,0.33500,0.19875,0.51125,0.33500,0.26375,0.16875,0.46375,0.33875,0.40875,0.33750,1


## Data cleaning

In [5]:
df1=df.dropna().drop_duplicates()
df1.shape

(3719, 17)

In [6]:
# Checking that we don't have any null values
assert df1.isnull().all().all() == False

## Non-pipline steps

In [7]:
features_df = df1.drop("label", axis=1)
label_df = df1["label"]

In [8]:

from sklearn.model_selection import train_test_split
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(features_df, label_df, test_size=0.2, random_state=10)

In [9]:
x_train_df.shape, x_test_df.shape, y_train_df.shape, y_test_df.shape 

((2975, 16), (744, 16), (2975,), (744,))

In [10]:
x=df1.copy()
#x=x.drop(["label"], axis = 1)
x_cols = ['leftShoulder_x', 'rightShoulder_x',
        'leftElbow_x', 'rightElbow_x',
        'leftWrist_x', 'rightWrist_x',
        'leftHip_x', 'rightHip_x']
#xtrans = XCentralizer(x_cols)
#x = xtrans.transform(x)

y_cols = list(set(x.columns)-set(x_cols))
#print(y_cols)
#ytrans = YCentralizer(y_cols)
#x = ytrans.transform(x)

#ytrans = YScaler()
#x = ytrans.transform(x)
x[:2]

,leftShoulder_x,leftShoulder_y,rightShoulder_x,rightShoulder_y,leftElbow_x,leftElbow_y,rightElbow_x,rightElbow_y,leftWrist_x,leftWrist_y,rightWrist_x,rightWrist_y,leftHip_x,leftHip_y,rightHip_x,rightHip_y,label
0,0.4925,0.18750,0.4000,0.18250,0.5050,0.26000,0.34375,0.1950,0.50875,0.33875,0.26625,0.16875,0.4650,0.34375,0.41125,0.34625,1
1,0.4925,0.18875,0.4025,0.18625,0.5075,0.25875,0.33250,0.1975,0.50000,0.33750,0.27625,0.17500,0.4675,0.33625,0.40875,0.33750,1


In [11]:
from sklearn.pipeline import make_pipeline, make_union

processing_pipeline = make_pipeline(
    XCentralizer(x_cols),
    YCentralizer(y_cols), 
    YScaler(),
    Shuffler()
    )

In [12]:
processed_df = processing_pipeline.fit_transform(x)

In [13]:
%%run_pytest[clean]
def test_processingpipeline():
    # remember, this first pipeline only acts on the features, not the target.
    processed_df = processing_pipeline.fit_transform(x)
    
    # check for data leakage
    assert x.shape[0] == processed_df.shape[0]

============================= test session starts ==============================
platform darwin -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
rootdir: /Users/lsafari/drone_steering/models, inifile:
plugins: remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collected 1 item

drone_pos_model.py .                                                     [100%]

=========================== 1 passed in 0.03 seconds ===========================


In [14]:
df_train = processed_df.iloc[:int(processed_df.shape[0]*0.8)]
df_val = processed_df.iloc[int(processed_df.shape[0]*0.8):]

In [15]:
assert df_train.shape[0] + df_val.shape[0] == processed_df.shape[0]

In [16]:
#y_train_df = df_train['label']
#x_train_df = df_train.drop('label', axis = 1)
#y_val_df = df_val['label']
#x_val_df = df_val.drop('label', axis = 1)
#x_train=x_train_df.values
#y_train=y_train_df.values
#x_val=x_val_df.values
#y_val=y_val_df.values

In [17]:
#x_train
#print("x_train=", x_train[0],"\n y_train=", y_train[0])

# Training Model

In [18]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras.constraints import maxnorm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from math import sqrt
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [19]:
from keras.utils import to_categorical
#y_train = to_categorical(y_train)
#y_val = to_categorical(y_val)

# Model Architecture

In [20]:
from keras import models, layers
from keras.models import Model
from keras.layers import Input, Dense
from keras import optimizers, losses, metrics

def create_model():
    #default vaues
    #activation="relu"
    #optimizer="adam"
    lr=0.01
    #momentum=0
    #creat model
    model = models.Sequential()
    model.add(layers.Dense(
        32, 
        activation="relu", 
        input_shape=(16, )))
    model.add(layers.Dense(15, activation="relu"))
    model.add(layers.Dense(5, activation="softmax")) #is a fast rectifier
    #model.summary()   

    model.compile(
    optimizer=optimizers.RMSprop(lr=0.01),
    loss=losses.categorical_crossentropy,
    metrics=["accuracy"] 
    )
    return model

In [22]:
#implements the Scikit-Learn classifier interface
#build_fn: callable function or class instance, sk_params: model parameters & fitting parameters
model_classifier = KerasClassifier(build_fn=create_model, epochs = 1000, batch_size = 16)
type(model_classifier)
#model_classifier.fit()

keras.wrappers.scikit_learn.KerasClassifier

In [23]:
finalpipeline = (make_pipeline(processing_pipeline,model_classifier))
history = finalpipeline.fit(x_train_df, y_train_df)
type(history)

Epoch 1/1000
2975/2975 [==============================] - 0s 124us/step - loss: 1.5748 - acc: 0.2487
Epoch 2/1000
2975/2975 [==============================] - 0s 71us/step - loss: 1.5661 - acc: 0.2679
Epoch 3/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5656 - acc: 0.2820
Epoch 4/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5644 - acc: 0.2770
Epoch 5/1000
2975/2975 [==============================] - 0s 71us/step - loss: 1.5647 - acc: 0.2703
Epoch 6/1000
2975/2975 [==============================] - 0s 72us/step - loss: 1.5647 - acc: 0.2706
Epoch 7/1000
2975/2975 [==============================] - 0s 71us/step - loss: 1.5629 - acc: 0.2689
Epoch 8/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5641 - acc: 0.2756
Epoch 9/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5613 - acc: 0.2763
Epoch 10/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5631 - acc: 0.272

2975/2975 [==============================] - 0s 68us/step - loss: 1.5579 - acc: 0.2743
Epoch 83/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5639 - acc: 0.2733
Epoch 84/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5557 - acc: 0.2723
Epoch 85/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5623 - acc: 0.2797
Epoch 86/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5665 - acc: 0.2635
Epoch 87/1000
2975/2975 [==============================] - 0s 67us/step - loss: 1.5645 - acc: 0.2736
Epoch 88/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5578 - acc: 0.2723
Epoch 89/1000
2975/2975 [==============================] - 0s 72us/step - loss: 1.5679 - acc: 0.2750
Epoch 90/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5603 - acc: 0.2753
Epoch 91/1000
2975/2975 [==============================] - 0s 72us/step - loss: 1.5643 - acc: 0.2696
Epoc

2975/2975 [==============================] - 0s 64us/step - loss: 1.5603 - acc: 0.2803
Epoch 163/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5614 - acc: 0.2746
Epoch 164/1000
2975/2975 [==============================] - 0s 72us/step - loss: 1.5625 - acc: 0.2760
Epoch 165/1000
2975/2975 [==============================] - 0s 72us/step - loss: 1.5725 - acc: 0.2803
Epoch 166/1000
2975/2975 [==============================] - 0s 72us/step - loss: 1.5599 - acc: 0.2709
Epoch 167/1000
2975/2975 [==============================] - 0s 71us/step - loss: 1.5574 - acc: 0.2760
Epoch 168/1000
2975/2975 [==============================] - 0s 74us/step - loss: 1.5659 - acc: 0.2776
Epoch 169/1000
2975/2975 [==============================] - 0s 75us/step - loss: 1.5591 - acc: 0.2743
Epoch 170/1000
2975/2975 [==============================] - 0s 77us/step - loss: 1.5611 - acc: 0.2810
Epoch 171/1000
2975/2975 [==============================] - 0s 73us/step - loss: 1.5708 - acc: 0.

2975/2975 [==============================] - 0s 71us/step - loss: 1.5610 - acc: 0.2790
Epoch 243/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5643 - acc: 0.2723
Epoch 244/1000
2975/2975 [==============================] - 0s 68us/step - loss: 1.5597 - acc: 0.2639
Epoch 245/1000
2975/2975 [==============================] - 0s 71us/step - loss: 1.5609 - acc: 0.2686
Epoch 246/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5629 - acc: 0.2770
Epoch 247/1000
2975/2975 [==============================] - 0s 64us/step - loss: 1.5602 - acc: 0.2635
Epoch 248/1000
2975/2975 [==============================] - 0s 66us/step - loss: 1.5655 - acc: 0.2736
Epoch 249/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5627 - acc: 0.2696
Epoch 250/1000
2975/2975 [==============================] - 0s 68us/step - loss: 1.5640 - acc: 0.2723
Epoch 251/1000
2975/2975 [==============================] - 0s 70us/step - loss: 1.5593 - acc: 0.

2975/2975 [==============================] - 0s 76us/step - loss: 1.5793 - acc: 0.2696
Epoch 323/1000
2975/2975 [==============================] - 0s 74us/step - loss: 1.5597 - acc: 0.2659
Epoch 324/1000
2975/2975 [==============================] - 0s 75us/step - loss: 1.5623 - acc: 0.2739
Epoch 325/1000
2975/2975 [==============================] - 0s 76us/step - loss: 1.5591 - acc: 0.2753
Epoch 326/1000
2975/2975 [==============================] - 0s 75us/step - loss: 1.5629 - acc: 0.2726
Epoch 327/1000
2975/2975 [==============================] - 0s 74us/step - loss: 1.5611 - acc: 0.2655
Epoch 328/1000
2975/2975 [==============================] - 0s 76us/step - loss: 1.5644 - acc: 0.2760
Epoch 329/1000
2975/2975 [==============================] - 0s 74us/step - loss: 1.5740 - acc: 0.2692
Epoch 330/1000
2975/2975 [==============================] - 0s 74us/step - loss: 1.5664 - acc: 0.2756
Epoch 331/1000
2975/2975 [==============================] - 0s 77us/step - loss: 1.5656 - acc: 0.

...

2975/2975 [==============================] - 0s 84us/step - loss: 1.5589 - acc: 0.2703
Epoch 723/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5584 - acc: 0.2733
Epoch 724/1000
2975/2975 [==============================] - 0s 83us/step - loss: 1.5588 - acc: 0.2746
Epoch 725/1000
2975/2975 [==============================] - 0s 85us/step - loss: 1.5586 - acc: 0.2776
Epoch 726/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5589 - acc: 0.2692
Epoch 727/1000
2975/2975 [==============================] - 0s 85us/step - loss: 1.5592 - acc: 0.2733
Epoch 728/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5586 - acc: 0.2770
Epoch 729/1000
2975/2975 [==============================] - 0s 85us/step - loss: 1.5587 - acc: 0.2699
Epoch 730/1000
2975/2975 [==============================] - 0s 77us/step - loss: 1.5590 - acc: 0.2739
Epoch 731/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5585 - acc: 0.

2975/2975 [==============================] - 0s 84us/step - loss: 1.5589 - acc: 0.2692
Epoch 803/1000
2975/2975 [==============================] - 0s 83us/step - loss: 1.5584 - acc: 0.2807
Epoch 804/1000
2975/2975 [==============================] - 0s 83us/step - loss: 1.5584 - acc: 0.2746
Epoch 805/1000
2975/2975 [==============================] - 0s 83us/step - loss: 1.5591 - acc: 0.2666
Epoch 806/1000
2975/2975 [==============================] - 0s 83us/step - loss: 1.5588 - acc: 0.2733
Epoch 807/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5588 - acc: 0.2716
Epoch 808/1000
2975/2975 [==============================] - 0s 83us/step - loss: 1.5592 - acc: 0.2750
Epoch 809/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5589 - acc: 0.2739
Epoch 810/1000
2975/2975 [==============================] - 0s 80us/step - loss: 1.5590 - acc: 0.2615
Epoch 811/1000
2975/2975 [==============================] - 0s 79us/step - loss: 1.5584 - acc: 0.

2975/2975 [==============================] - 0s 94us/step - loss: 1.5590 - acc: 0.2726
Epoch 883/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5589 - acc: 0.2713
Epoch 884/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5585 - acc: 0.2787
Epoch 885/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5592 - acc: 0.2571
Epoch 886/1000
2975/2975 [==============================] - 0s 92us/step - loss: 1.5590 - acc: 0.2743
Epoch 887/1000
2975/2975 [==============================] - 0s 92us/step - loss: 1.5588 - acc: 0.2733
Epoch 888/1000
2975/2975 [==============================] - 0s 90us/step - loss: 1.5586 - acc: 0.2692
Epoch 889/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5585 - acc: 0.2682
Epoch 890/1000
2975/2975 [==============================] - 0s 87us/step - loss: 1.5591 - acc: 0.2773
Epoch 891/1000
2975/2975 [==============================] - 0s 91us/step - loss: 1.5590 - acc: 0.

2975/2975 [==============================] - 0s 93us/step - loss: 1.5585 - acc: 0.2783
Epoch 963/1000
2975/2975 [==============================] - 0s 88us/step - loss: 1.5590 - acc: 0.2645
Epoch 964/1000
2975/2975 [==============================] - 0s 90us/step - loss: 1.5591 - acc: 0.2713
Epoch 965/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5590 - acc: 0.2615
Epoch 966/1000
2975/2975 [==============================] - 0s 92us/step - loss: 1.5588 - acc: 0.2716
Epoch 967/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5590 - acc: 0.2645
Epoch 968/1000
2975/2975 [==============================] - 0s 88us/step - loss: 1.5589 - acc: 0.2699
Epoch 969/1000
2975/2975 [==============================] - 0s 84us/step - loss: 1.5588 - acc: 0.2739
Epoch 970/1000
2975/2975 [==============================] - 0s 88us/step - loss: 1.5590 - acc: 0.2686
Epoch 971/1000
2975/2975 [==============================] - 0s 89us/step - loss: 1.5590 - acc: 0.

sklearn.pipeline.Pipeline

# Grid Search

In [27]:
print(finalpipeline.get_params().keys(),'\n')
print(model_classifier.get_params().keys(),'\n')
print(model_classifier.get_params()["epochs"])

dict_keys(['memory', 'steps', 'pipeline', 'kerasclassifier', 'pipeline__memory', 'pipeline__steps', 'pipeline__xcentralizer', 'pipeline__ycentralizer', 'pipeline__yscaler', 'pipeline__shuffler', 'pipeline__xcentralizer__x_columns', 'pipeline__ycentralizer__y_columns', 'kerasclassifier__epochs', 'kerasclassifier__batch_size', 'kerasclassifier__build_fn']) 

dict_keys(['epochs', 'batch_size', 'build_fn']) 

20


In [28]:
#Use sckit_learn to grid search
activation = ['relu']
lr = [0.0001, 0.001, 0.1]
neurons = [1, 5, 10, 15, 20, 25, 30]
optimizer = ['RMSprop','Adam'] #['SGD', 'Adgrad', 'Adadelta', 'Adamax', 'Nadam']
######## grid serach epochs, batch_size
epochs = [100]
batch_size = [20]
#param_grid = dict(epochs=epochs, batch_size=batch_size)
param_grid = {"kerasclassifier__epochs": epochs, "kerasclassifier__batch_size": batch_size}


grid_search = GridSearchCV(estimator=finalpipeline, param_grid=param_grid, n_jobs=-1)

In [29]:
grid_result = grid_search.fit(x_train_df, y_train_df)

Epoch 1/100
Epoch 1/100
Epoch 1/100


KeyboardInterrupt: 

In [ ]:
grid_result.cv_results_

In [ ]:
##############################################################
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
history = model_classifier.fit(x_train, y_train, epochs=100, batch_size=30)

In [24]:

plt.plot(history.history["loss"], label=["loss"]) #play with hyperparameters to see the changes
plt.legend()
plt.show()
plt.close()

plt.plot(history.history["acc"],  label=["acc"])
plt.legend()
plt.show()
plt.close()



AttributeError: 'Pipeline' object has no attribute 'history'

In [ ]:
test_loss, test_acc = model_classifier.evaluate(x_val, y_val)
print("Loss / Accuracy Evaluation")
print("--------------------------")
print("Loss:     " + str(round(test_loss,5)))
print("Accuracy: " + str(round(test_acc,5)))

In [ ]:
val_y_pred = model_classifier.predict(x_val)

In [ ]:
val_y_pred[10]

In [ ]:
y_val[10]

### Save Model in tensorflow.js Format

The tensorflowjs library can't be installed directly with pip / conda due to conflicting dependencies. Best is to set up a new environment explicitly for this and install tensorflowjs with the following commands:

```
pip install tensorflow==1.11.0rc2 h5py numpy keras
pip install --no-deps tensorflowjs
```

In [ ]:
! pip install tensorflow==1.11.0rc2 h5py numpy keras
! pip install --no-deps tensorflowjs

In [ ]:
import tensorflowjs as tfjs

In [ ]:
tfjs.converters.save_keras_model(model, 'model_tfjs')

We need to adapt the two files as follows in order for them to work on Azure:
* add a file extension .pb to the file with no extension (otherwise Azure blocks it from viewing)
* adapt the automatically generated model.json to reflect the extension .pb